In [1]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments
import pandas as pd
from sklearn.model_selection import train_test_split


C:\Users\rente\anaconda3\Lib\site-packages\transformers\utils\generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
C:\Users\rente\anaconda3\Lib\site-packages\transformers\utils\generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


In [2]:
data_path = "C:\\Users\\rente\\Desktop\\Proiect_Licenta\\baza_stiri_clean.xlsx"
data = pd.read_excel(data_path)


In [3]:
# Combinarea coloanelor "Titlu", "Lead" și "Conținut" într-un singur text
data['text'] = data['Titlu'] + " [SEP] " + data['Lead'] + " [SEP] " + data['Continut']
texts = data['text'].tolist()

In [4]:
# Împărțirea textelor în seturi de antrenare și validare
train_texts, val_texts = train_test_split(texts, test_size=0.1)


In [5]:
# Salvarea textelor în fișiere pentru a putea fi încărcate de TextDataset
with open('train_texts.txt', 'w', encoding='utf-8') as f:
    f.write("\n".join(train_texts))
with open('val_texts.txt', 'w', encoding='utf-8') as f:
    f.write("\n".join(val_texts))


In [6]:
# Încărcarea tokenizer-ului specific modelului GPT-2
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')


In [7]:
# Pre-procesarea datelor
train_dataset = TextDataset(
    tokenizer=tokenizer,
    file_path="train_texts.txt",
    block_size=128)

val_dataset = TextDataset(
    tokenizer=tokenizer,
    file_path="val_texts.txt",
    block_size=128)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False)


C:\Users\rente\anaconda3\Lib\site-packages\transformers\data\datasets\language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [8]:
# Inițializarea modelului GPT-2
model = GPT2LMHeadModel.from_pretrained('gpt2')

In [9]:
# Setarea argumentelor de antrenare
training_args = TrainingArguments(
    output_dir='./results2',
    overwrite_output_dir=True,
    num_train_epochs=15, 
    per_device_train_batch_size=4, 
    per_device_eval_batch_size=4,
    save_steps=10_000,
    save_total_limit=2,
    prediction_loss_only=True,
)


In [10]:
# Inițializarea Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)


In [11]:
# Antrenarea modelului
trainer.train()

Step,Training Loss
500,4.862000
1000,4.172600
1500,3.909800
2000,3.758800
2500,3.667200
3000,3.567400
3500,3.478700
4000,3.436700
4500,3.386500
5000,3.337800


TrainOutput(global_step=303645, training_loss=2.1913005372380203, metrics={'train_runtime': 28178.16, 'train_samples_per_second': 43.102, 'train_steps_per_second': 10.776, 'total_flos': 7.933707952128e+16, 'train_loss': 2.1913005372380203, 'epoch': 15.0})

In [12]:
# Procesul de creare al locatiei de salvare al modelului
model_path = './model_salvat2'

In [13]:
#salvarea propriu zisa 
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)


('./model_salvat2\\tokenizer_config.json',
 './model_salvat2\\special_tokens_map.json',
 './model_salvat2\\vocab.json',
 './model_salvat2\\merges.txt',
 './model_salvat2\\added_tokens.json')

In [14]:
# Re-încărcarea tokenizer-ului și a modelului antrenat
tokenizer = GPT2Tokenizer.from_pretrained(model_path)
model = GPT2LMHeadModel.from_pretrained(model_path)

# Cu ce cuvant vrei sa fie construita stirea 
prompt_text = "Lege noua financiara"

# Encodarea promptului pentru a fi compatibil cu inputul modelului
input_ids = tokenizer.encode(prompt_text, return_tensors='pt')

# Argumente folosite in constructia textului 
generated_outputs = model.generate(
    input_ids,
    max_length=600,  # Lungimea maximă a textului generat
    num_beams=2,  # Activarea Beam Search cu 5 module, functia de activare al Beam Search ului sa nu se mai repede cuvintele
    temperature=0.7,  # Ajustarea temperaturii pentru diversitate
    no_repeat_ngram_size=2,  # Prevenirea repetițiilor 
    early_stopping=True,  # Oprirea anticipată dacă toate modulele de la beam ajung la tokenul de stop
    num_return_sequences=2,  # Numărul de secvențe generate
)

# Decodarea și afișarea textului generat
generated_text = tokenizer.decode(generated_outputs[0], skip_special_tokens=True)
print("Text generat:")
print(generated_text)

C:\Users\rente\anaconda3\Lib\site-packages\transformers\generation\configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Text generat:
Lege noua financiara a pensiilor a fost adoptata de Senat, care a adoptat o lege care prevede ca, incepand cu 1 ianuarie 2024, cei care beneficiaza de servicii de asistenta sociala, sa beneficieze de o contributie de 10% din veniturile salariale acordate persoanelor fizice si juridice, precum si de contributia la asigurarile sociale de sanatate, pensie sau alte cheltuieli publice sa fie beneficiarii de aceeasi suma.In ceea ce priveste salariile, s-a stabilit ca in anul 2024 contributiile la pilonul II de pensii vor creste cu 13,8%, iar in cresterectie, de la 1.785 lei,6% in 2023, fata in cazul de cat era calculatia precedentul prezentata, si la pensia actualul precedenta din 2026, i-2023.Citeste modificarea bugetul pentru contributivului de impozitare voluntara de statutare a contributivelor social-contributivarelor de sectoriale va fi stabilita in luna in 2024-cu contributii sectorul militar voluntar, a cresedatiei voluntariat, drei sociali si a beneficiar sector 3, cu 1

In [15]:
def generate_texts(model, tokenizer, seed_text, num_generations=5):
    generated_texts = []
    input_ids = tokenizer.encode(seed_text, return_tensors='pt')

    for _ in range(num_generations):
        # Generarea textului
        generated_outputs = model.generate(
            input_ids,
            max_length=600,  # Adjust the max length based on your needs
            num_beams=5,
            do_sample = True,
            no_repeat_ngram_size=2,
            num_return_sequences=1,
            pad_token_id=tokenizer.eos_token_id,
        )

        generated_text = tokenizer.decode(generated_outputs[0], skip_special_tokens=True)
        generated_texts.append(generated_text)

    return generated_texts

# Încărcarea tokenizer-ului și a modelului antrenat
model_path = './model_salvat'  # Update this path to your model's path
tokenizer = GPT2Tokenizer.from_pretrained(model_path)
model = GPT2LMHeadModel.from_pretrained(model_path)

# Aici setezi seed-ul de cuvinte
seed_text = "Lege noua "
generated_texts = generate_texts(model, tokenizer, seed_text, num_generations=10)

def save_to_excel(texts, file_name="generated_texts.xlsx"):
    df = pd.DataFrame(texts, columns=["Generated Text"])
    df.to_excel(file_name, index=False)

# Salvarea textelor generate într-un fișier Excel
save_to_excel(generated_texts, "generated_texts.xlsx")